<a href="https://colab.research.google.com/github/Nithin2211/Quantam-Enhanced/blob/main/Quantum_enhanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# Upload files to Colab
uploaded = files.upload()  # Select the 3 CSV files

# This will upload:
# - fer2013_sample.csv
# - pie_trajectory_sample.csv
# - combined_emotion_trajectory_sample.csv


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np

# Load FER-2013 dataset
fer2013_df = pd.read_csv('fer2013_sample.csv')
print(fer2013_df.head())

# Load PIE dataset
pie_df = pd.read_csv('pie_trajectory_sample.csv')
print(pie_df.head())

# Load combined dataset
combined_df = pd.read_csv('combined_emotion_trajectory_sample.csv')
print(combined_df.head())


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Parse FER-2013 images
X_train = []
y_train = []

for idx, row in fer2013_df.iterrows():
    emotion = int(row['emotion'])
    pixels = np.array([int(p) for p in row['pixels'].split()], dtype=np.float32) / 255.0
    img = pixels.reshape(48, 48, 1)  # Reshape to 48x48 grayscale

    X_train.append(img)
    y_train.append(emotion)

X_train = np.array(X_train)
y_train = np.array(y_train)
print(f"Emotion data shape: {X_train.shape}")  # (500, 48, 48, 1)

# Parse PIE trajectories
trajectories = []
emotions = []

for idx, row in pie_df.iterrows():
    emotion = int(row['emotion'])
    traj_frames = row['trajectory'].split('|')

    traj_sequence = []
    for frame_str in traj_frames:
        x, y, w, h = map(float, frame_str.split('_'))
        traj_sequence.append([x, y, w, h])

    trajectories.append(np.array(traj_sequence))
    emotions.append(emotion)

trajectories = np.array(trajectories)
emotions = np.array(emotions)
print(f"Trajectory data shape: {trajectories.shape}")  # (500, 30, 4)


**Section 1: Install Dependencies**

In [ ]:
!pip install -q tensorflow keras scikit-learn numpy matplotlib opencv-python pillow pandas scipy
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
import cv2
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
from scipy import stats

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)

print("✅ All dependencies installed!")


**Section 2: Load and Parse Uploaded Datasets**

In [ ]:
print("\n" + "="*70)
print("LOADING UPLOADED DATASETS")
print("="*70)

# Load the CSV files
fer2013_df = pd.read_csv('fer2013_sample.csv')
pie_df = pd.read_csv('pie_trajectory_sample.csv')
combined_df = pd.read_csv('combined_emotion_trajectory_sample.csv')

print(f"✅ FER-2013 loaded: {fer2013_df.shape}")
print(f"✅ PIE loaded: {pie_df.shape}")
print(f"✅ Combined loaded: {combined_df.shape}")

# Emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']


*Section 3: Parse FER-2013 Emotion Data*

In [ ]:
print("\n" + "="*70)
print("PARSING FER-2013 EMOTION DATA")
print("="*70)

X_train_fer = []
y_train_fer = []

print("🔄 Parsing FER-2013 images...")

for idx, row in fer2013_df.iterrows():
    emotion = int(row['emotion'])

    # Parse pixel string into numpy array
    pixels = np.array([int(p) for p in row['pixels'].split()], dtype=np.float32)

    # Normalize to 0-1 range
    pixels = pixels / 255.0

    # Reshape to 48x48x1 (grayscale image)
    img = pixels.reshape(48, 48, 1)

    X_train_fer.append(img)
    y_train_fer.append(emotion)

X_train_fer = np.array(X_train_fer)
y_train_fer = np.array(y_train_fer)

print(f"✅ FER-2013 Images shape: {X_train_fer.shape}")
print(f"   Emotions shape: {y_train_fer.shape}")
print(f"\n📊 Emotion distribution:")
for emotion_idx, label in enumerate(emotion_labels):
    count = (y_train_fer == emotion_idx).sum()
    print(f"     {label:12} : {count:3d} samples")

# Split into train and test (80/20)
X_fer_train, X_fer_test, y_fer_train, y_fer_test = train_test_split(
    X_train_fer, y_train_fer, test_size=0.2, random_state=42, stratify=y_train_fer
)

print(f"\n✅ Train/Test split: {X_fer_train.shape[0]} train / {X_fer_test.shape[0]} test")


**Section 4: Parse PIE Trajectory Data**

In [ ]:
print("\n" + "="*70)
print("PARSING PIE TRAJECTORY DATA")
print("="*70)

X_trajectories = []
y_emotions_pie = []

print("🔄 Parsing PIE trajectories...")

for idx, row in pie_df.iterrows():
    emotion = int(row['emotion'])

    # Parse trajectory frames (pipe-separated)
    traj_frames = row['trajectory'].split('|')

    # Parse each frame (x_y_w_h format)
    trajectory_sequence = []
    for frame_str in traj_frames:
        x, y, w, h = map(float, frame_str.split('_'))
        trajectory_sequence.append([x, y, w, h])

    X_trajectories.append(np.array(trajectory_sequence, dtype=np.float32))
    y_emotions_pie.append(emotion)

X_trajectories = np.array(X_trajectories)
y_emotions_pie = np.array(y_emotions_pie)

print(f"✅ Trajectories shape: {X_trajectories.shape}")
print(f"   Format: (sequences, frames, features) = (500, 30, 4)")
print(f"✅ Emotions shape: {y_emotions_pie.shape}")

# Normalize trajectories for LSTM
X_trajectories_norm = (X_trajectories - X_trajectories.mean(axis=(0, 1))) / (X_trajectories.std(axis=(0, 1)) + 1e-6)

print(f"✅ Trajectories normalized (mean=0, std=1)")

# Split into train and test (80/20)
X_traj_train, X_traj_test, y_traj_train, y_traj_test = train_test_split(
    X_trajectories_norm, y_emotions_pie, test_size=0.2, random_state=42, stratify=y_emotions_pie
)

print(f"✅ Trajectory Train/Test split: {X_traj_train.shape[0]} train / {X_traj_test.shape[0]} test")


**Section 5: Build and Train CNN Emotion Recognition Model**

In [ ]:
print("\n" + "="*70)
print("BUILDING CNN EMOTION RECOGNITION MODEL")
print("="*70)

def build_emotion_cnn_model(input_shape=(48, 48, 1), num_classes=7):
    """
    Build CNN for emotion recognition (MobileNetV2-inspired architecture)
    """
    model = models.Sequential([
        # Input layer
        layers.Input(shape=input_shape),

        # Block 1
        layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Block 2: Depthwise Separable
        layers.DepthwiseConv2D((3, 3), padding='same', activation='relu'),
        layers.Conv2D(64, (1, 1), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Block 3
        layers.DepthwiseConv2D((3, 3), padding='same', activation='relu'),
        layers.Conv2D(128, (1, 1), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Block 4
        layers.DepthwiseConv2D((3, 3), padding='same', activation='relu'),
        layers.Conv2D(256, (1, 1), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Global pooling
        layers.GlobalAveragePooling2D(),

        # Dense layers
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),

        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),

        # Output
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

# Build emotion model
emotion_model = build_emotion_cnn_model(input_shape=(48, 48, 1), num_classes=7)

emotion_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ CNN Model built!")
print(emotion_model.summary())

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    fill_mode='nearest'
)

# Train emotion model
print("\n🔄 Training CNN Emotion Recognition Model...")
history_cnn = emotion_model.fit(
    train_datagen.flow(X_fer_train, y_fer_train, batch_size=16),
    epochs=15,
    validation_data=(X_fer_test, y_fer_test),
    verbose=1
)

# Evaluate
y_pred_fer = emotion_model.predict(X_fer_test, verbose=0).argmax(axis=1)
emotion_accuracy = accuracy_score(y_fer_test, y_pred_fer)
emotion_precision = precision_score(y_fer_test, y_pred_fer, average='weighted', zero_division=0)
emotion_recall = recall_score(y_fer_test, y_pred_fer, average='weighted', zero_division=0)
emotion_f1 = f1_score(y_fer_test, y_pred_fer, average='weighted', zero_division=0)

print(f"\n✅ CNN Emotion Model Performance:")
print(f"   Accuracy:  {emotion_accuracy:.4f}")
print(f"   Precision: {emotion_precision:.4f}")
print(f"   Recall:    {emotion_recall:.4f}")
print(f"   F1-Score:  {emotion_f1:.4f}")

**Section 6: Build and Train LSTM Trajectory Prediction Model**

In [ ]:
print("\n" + "="*70)
print("BUILDING LSTM TRAJECTORY PREDICTION MODEL")
print("="*70)

def build_lstm_trajectory_model(seq_length=30, num_features=4, num_classes=7):
    """
    LSTM model for trajectory prediction with emotion auxiliary task
    """
    trajectory_input = layers.Input(shape=(seq_length, num_features), name='trajectory_input')

    # LSTM layers
    x = layers.LSTM(128, return_sequences=True, activation='relu')(trajectory_input)
    x = layers.Dropout(0.2)(x)

    x = layers.LSTM(64, return_sequences=True, activation='relu')(x)
    x = layers.Dropout(0.2)(x)

    x = layers.LSTM(32, return_sequences=False, activation='relu')(x)
    x = layers.Dropout(0.2)(x)

    # Trajectory prediction head (predict next 5 frames)
    trajectory_pred = layers.Dense(20, name='trajectory_prediction')(x)  # 5 frames * 4 features

    # Emotion classification head (auxiliary task)
    emotion_branch = layers.Dense(64, activation='relu')(x)
    emotion_branch = layers.Dense(7, activation='softmax', name='emotion_prediction')(emotion_branch)

    model = models.Model(
        inputs=trajectory_input,
        outputs=[trajectory_pred, emotion_branch]
    )

    return model

# Build LSTM model
lstm_model = build_lstm_trajectory_model(seq_length=30, num_features=4, num_classes=7)

lstm_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={
        'trajectory_prediction': 'mse',
        'emotion_prediction': 'sparse_categorical_crossentropy'
    },
    loss_weights={
        'trajectory_prediction': 1.0,
        'emotion_prediction': 0.5
    },
    metrics={
        'trajectory_prediction': 'mae',
        'emotion_prediction': 'accuracy'
    }
)

print(" LSTM Model built!")
print(lstm_model.summary())

# Ensure we have the correct 1D emotion labels for the train/test sets.
# (Re-perform the split for the labels to guarantee correct shape, as
# y_traj_train from a previous run appears to have an incorrect shape in the kernel state).
# We use np.arange as dummy features to get the correct label splits.
_, _, y_emotion_train_labels, y_emotion_test_labels = train_test_split(
    np.arange(len(y_emotions_pie)),
    y_emotions_pie,
    test_size=0.2,
    random_state=42,
    stratify=y_emotions_pie
)

# Prepare target trajectories (predict last 5 frames of input sequence)
# Each sequence is 30 frames. Last 5 frames (5*4=20 features) as target.
y_traj_prediction_train = X_traj_train[:, -5:, :].reshape(len(X_traj_train), -1)
y_traj_prediction_test = X_traj_test[:, -5:, :].reshape(len(X_traj_test), -1)

print(f"\n Target shapes:")
print(f"   y_traj_prediction_train: {y_traj_prediction_train.shape}")
print(f"   y_traj_prediction_test: {y_traj_prediction_test.shape}")
print(f"   y_emotion_train_labels: {y_emotion_train_labels.shape}")
print(f"   y_emotion_test_labels: {y_emotion_test_labels.shape}")

# Train LSTM
print("\n Training LSTM Trajectory Prediction Model...")
lstm_history = lstm_model.fit(
    X_traj_train,
    {
        'trajectory_prediction': y_traj_prediction_train,
        'emotion_prediction': y_emotion_train_labels # Use the explicitly defined labels
    },
    epochs=15,
    batch_size=16,
    validation_data=(
        X_traj_test,
        {
            'trajectory_prediction': y_traj_prediction_test,
            'emotion_prediction': y_emotion_test_labels # Use the explicitly defined labels
        }
    ),
    verbose=1
)

print(" LSTM Model training completed!")

** Section 7: Emotion-to-Vehicle Behavior Mapping**

In [ ]:
print("\n" + "="*70)
print("EMOTION-TO-VEHICLE BEHAVIOR MAPPING")
print("="*70)

class EmotionBehaviorMapper:
    """
    Maps emotions to vehicle actions (BRAKE, CAUTION, PROCEED)
    """

    def __init__(self):
        self.emotion_to_action = {
            0: {'action': 'BRAKE', 'speed_factor': 0.0, 'label': 'Angry'},
            1: {'action': 'BRAKE', 'speed_factor': 0.0, 'label': 'Disgust'},
            2: {'action': 'BRAKE', 'speed_factor': 0.0, 'label': 'Fear'},
            3: {'action': 'PROCEED', 'speed_factor': 1.0, 'label': 'Happy'},
            4: {'action': 'PROCEED', 'speed_factor': 1.0, 'label': 'Neutral'},
            5: {'action': 'CAUTION', 'speed_factor': 0.6, 'label': 'Sad'},
            6: {'action': 'CAUTION', 'speed_factor': 0.6, 'label': 'Surprise'}
        }

    def get_vehicle_action(self, emotion_idx, confidence=1.0):
        """Get vehicle action based on emotion"""
        action_info = self.emotion_to_action[emotion_idx]

        if confidence < 0.7:
            return {
                'action': 'CAUTION',
                'speed_factor': 0.6,
                'confidence': confidence,
                'label': f"{action_info['label']} (LOW CONF)"
            }

        return {
            'action': action_info['action'],
            'speed_factor': action_info['speed_factor'],
            'confidence': confidence,
            'label': action_info['label']
        }

    def adjust_trajectory(self, emotion_idx, current_trajectory, base_speed=50):
        """Adjust vehicle speed and path based on emotion"""
        action_info = self.get_vehicle_action(emotion_idx)

        new_speed = base_speed * action_info['speed_factor']

        if action_info['action'] == 'CAUTION':
            path_adjustment = 2.0
        elif action_info['action'] == 'BRAKE':
            path_adjustment = 0.0
        else:
            path_adjustment = 0.0

        return {
            'new_speed': new_speed,
            'path_adjustment': path_adjustment,
            'vehicle_action': action_info['action'],
            'emotion_label': action_info['label']
        }

emotion_mapper = EmotionBehaviorMapper()

print("\n📊 Emotion-to-Action Mapping Table:")
print("-" * 70)
print(f"{'Emotion':<15} {'Action':<10} {'Speed Factor':<12} {'Description':<35}")
print("-" * 70)
for emotion_idx, action_info in emotion_mapper.emotion_to_action.items():
    print(f"{action_info['label']:<15} {action_info['action']:<10} {action_info['speed_factor']:<12.2f}")
print("-" * 70)


**Section 8: Quantum-Inspired Probabilistic Decision Engine**

In [ ]:
print("\n" + "="*70)
print("QUANTUM-INSPIRED PROBABILISTIC DECISION ENGINE")
print("="*70)

class QuantumEmotionDecisionEngine:
    """
    Quantum-inspired logic for handling ambiguous emotional states.
    Uses probabilistic superposition and amplitude-based measurement.
    """

    def __init__(self, num_emotions=7):
        self.num_emotions = num_emotions
        self.action_space = ['BRAKE', 'CAUTION', 'PROCEED']
        self.emotion_action_map = {
            'BRAKE': [0, 1, 2],           # Angry, Disgust, Fear
            'CAUTION': [5, 6],            # Sad, Surprise
            'PROCEED': [3, 4]             # Happy, Neutral
        }

    def create_superposition_state(self, emotion_probabilities):
        """Create quantum superposition of emotional states"""
        amplitudes = np.sqrt(emotion_probabilities)
        amplitudes = amplitudes / np.linalg.norm(amplitudes)
        return amplitudes

    def compute_action_probabilities(self, emotion_probabilities, confidence):
        """Compute action probability using quantum logic"""
        amplitudes = self.create_superposition_state(emotion_probabilities)
        measurement_probs = amplitudes ** 2

        action_probs = {action: 0.0 for action in self.action_space}

        for emotion_idx, emotion_amp in enumerate(measurement_probs):
            for action, emotion_list in self.emotion_action_map.items():
                if emotion_idx in emotion_list:
                    action_probs[action] += emotion_amp

        total = sum(action_probs.values())
        if total > 0:
            action_probs = {k: v/total for k, v in action_probs.items()}

        # Confidence modifier
        if confidence > 0.8:
            max_action = max(action_probs, key=action_probs.get)
            action_probs = {k: 0.0 for k in action_probs.keys()}
            action_probs[max_action] = 1.0
        elif confidence < 0.5:
            uniform = {k: 1.0/len(action_probs) for k in action_probs.keys()}
            action_probs = {k: 0.5*action_probs[k] + 0.5*uniform[k] for k in action_probs.keys()}

        return action_probs

    def make_decision(self, emotion_probabilities, confidence_threshold=0.5):
        """Make final vehicle action decision"""
        max_confidence = np.max(emotion_probabilities)
        action_probs = self.compute_action_probabilities(emotion_probabilities, max_confidence)
        selected_action = max(action_probs, key=action_probs.get)

        return {
            'selected_action': selected_action,
            'action_probabilities': action_probs,
            'confidence': max_confidence,
            'is_uncertain': max_confidence < confidence_threshold
        }

quantum_engine = QuantumEmotionDecisionEngine(num_emotions=7)

print("✅ Quantum-inspired decision engine initialized!")

# Demo quantum logic
print("\n🔬 Quantum Decision Logic Examples:")
print("=" * 70)

# Example 1: Clear emotion
test_probs_clear = np.array([0.01, 0.01, 0.01, 0.95, 0.01, 0.01, 0.00])
decision_clear = quantum_engine.make_decision(test_probs_clear)
print(f"Clear Emotion (Happy, 95% confidence):")
print(f"  Selected Action: {decision_clear['selected_action']}")
print(f"  Action Probabilities: {decision_clear['action_probabilities']}")

print()

# Example 2: Ambiguous emotion
test_probs_ambiguous = np.array([0.20, 0.15, 0.25, 0.15, 0.15, 0.05, 0.05])
decision_ambiguous = quantum_engine.make_decision(test_probs_ambiguous)
print(f"Ambiguous Emotion (Mixed signals, ~40% max confidence):")
print(f"  Selected Action: {decision_ambiguous['selected_action']}")
print(f"  Action Probabilities: {decision_ambiguous['action_probabilities']}")
print(f"  Is Uncertain: {decision_ambiguous['is_uncertain']}")

**Section 9: Real-Time Prediction Pipeline**

In [ ]:
print("\n" + "="*70)
print("REAL-TIME PREDICTION PIPELINE")
print("="*70)

class RealTimePredictionPipeline:
    """
    Complete pipeline for AV perception and decision-making
    """

    def __init__(self, emotion_model, lstm_model, emotion_mapper, quantum_engine):
        self.emotion_model = emotion_model
        self.lstm_model = lstm_model
        self.emotion_mapper = emotion_mapper
        self.quantum_engine = quantum_engine

        self.frame_count = 0
        self.processing_times = []
        self.decisions_history = []

    def predict_frame(self, face_image, trajectory_sequence, use_quantum=True):
        """Process single frame and make vehicle decision"""
        import time
        start_time = time.time()

        # Step 1: Emotion recognition
        face_batch = np.expand_dims(face_image, axis=0)
        emotion_probs = self.emotion_model.predict(face_batch, verbose=0)[0]

        # Step 2: Trajectory prediction
        traj_batch = np.expand_dims(trajectory_sequence, axis=0)
        next_trajectory, emotion_aux = self.lstm_model.predict(traj_batch, verbose=0)

        # Step 3: Decide vehicle action
        if use_quantum:
            decision_result = self.quantum_engine.make_decision(emotion_probs)
            selected_emotion = np.argmax(emotion_probs)
            vehicle_action = decision_result['selected_action']
        else:
            selected_emotion = np.argmax(emotion_probs)
            action_info = self.emotion_mapper.get_vehicle_action(selected_emotion, emotion_probs[selected_emotion])
            vehicle_action = action_info['action']

        # Step 4: Adjust trajectory
        adjustment = self.emotion_mapper.adjust_trajectory(selected_emotion, trajectory_sequence, base_speed=50)

        processing_time = time.time() - start_time
        self.processing_times.append(processing_time)
        self.frame_count += 1

        result = {
            'frame_number': self.frame_count,
            'emotion_probabilities': emotion_probs,
            'detected_emotion': selected_emotion,
            'emotion_label': emotion_labels[selected_emotion],
            'confidence': emotion_probs[selected_emotion],
            'predicted_trajectory': next_trajectory,
            'vehicle_action': vehicle_action,
            'speed_adjustment': adjustment['new_speed'],
            'path_adjustment': adjustment['path_adjustment'],
            'processing_time_ms': processing_time * 1000,
            'use_quantum': use_quantum
        }

        self.decisions_history.append(result)
        return result

    def get_statistics(self):
        """Return processing statistics"""
        times_ms = np.array(self.processing_times) * 1000
        return {
            'total_frames_processed': self.frame_count,
            'avg_processing_time_ms': np.mean(times_ms),
            'max_processing_time_ms': np.max(times_ms),
            'min_processing_time_ms': np.min(times_ms),
            'real_time_capability': np.mean(times_ms) < 100
        }

# Initialize pipeline
pipeline = RealTimePredictionPipeline(
    emotion_model=emotion_model,
    lstm_model=lstm_model,
    emotion_mapper=emotion_mapper,
    quantum_engine=quantum_engine
)

print("✅ Real-time prediction pipeline initialized!")


**Section 10: Simulate Frame-by-Frame Processing**

In [ ]:
print("\n" + "="*70)
print("SIMULATING REAL-TIME FRAME-BY-FRAME PROCESSING")
print("="*70)

num_test_frames = 15
test_results = []

print(f"\n🎬 Processing {num_test_frames} frames...")

for frame_idx in range(num_test_frames):
    # Random test face and trajectory
    test_face = X_fer_test[np.random.randint(0, len(X_fer_test))]
    test_traj = X_traj_test[np.random.randint(0, len(X_traj_test))]

    # Process frame
    result = pipeline.predict_frame(test_face, test_traj, use_quantum=True)
    test_results.append(result)

    if (frame_idx + 1) % 5 == 0:
        print(f"  ✓ Processed {frame_idx + 1} frames...")

print(f"\n✅ Completed processing {len(test_results)} frames")

# Display sample results
print("\n" + "="*70)
print("Sample Frame Processing Results:")
print("="*70)

for i in [0, len(test_results)//2, -1]:
    r = test_results[i]
    print(f"\nFrame {r['frame_number']}:")
    print(f"  Detected Emotion: {r['emotion_label']} (confidence: {r['confidence']:.3f})")
    print(f"  Vehicle Action: {r['vehicle_action']}")
    print(f"  Speed Adjustment: {r['speed_adjustment']:.1f} km/h")
    print(f"  Processing Time: {r['processing_time_ms']:.2f} ms")

**Section 11: Evaluation Metrics**

In [ ]:
print("\n" + "="*70)
print("EVALUATION METRICS")
print("="*70)

emotions_predicted = [r['detected_emotion'] for r in test_results]
confidences = [r['confidence'] for r in test_results]
vehicle_actions = [r['vehicle_action'] for r in test_results]

stats = pipeline.get_statistics()

print(f"\n📊 Processing Statistics:")
print(f"  Total Frames Processed: {stats['total_frames_processed']}")
print(f"  Avg Processing Time: {stats['avg_processing_time_ms']:.2f} ms")
print(f"  Max Processing Time: {stats['max_processing_time_ms']:.2f} ms")
print(f"  Min Processing Time: {stats['min_processing_time_ms']:.2f} ms")
print(f"  Real-time Capability: {'✅ YES' if stats['real_time_capability'] else '❌ NO'} ({stats['avg_processing_time_ms']:.2f} ms < 100 ms required)")

print(f"\n📊 Emotion Recognition Statistics:")
print(f"  Average Confidence: {np.mean(confidences):.4f}")
print(f"  Confidence Std Dev: {np.std(confidences):.4f}")
print(f"  High Confidence (>0.7): {sum(1 for c in confidences if c > 0.7)} frames")
print(f"  Medium Confidence (0.5-0.7): {sum(1 for c in confidences if 0.5 <= c <= 0.7)} frames")
print(f"  Low Confidence (<0.5): {sum(1 for c in confidences if c < 0.5)} frames")

action_counts = pd.Series(vehicle_actions).value_counts()
print(f"\n🚗 Vehicle Action Distribution:")
for action, count in action_counts.items():
    print(f"  {action}: {count} frames ({100*count/len(vehicle_actions):.1f}%)")


**Section 12: Visualization**

In [ ]:
print("\n" + "="*70)
print("CREATING VISUALIZATIONS")
print("="*70)

fig = plt.figure(figsize=(16, 12))

# Plot 1: Emotion Model Training Accuracy
ax1 = plt.subplot(3, 3, 1)
ax1.plot(history_cnn.history['accuracy'], label='Train Accuracy')
ax1.plot(history_cnn.history['val_accuracy'], label='Validation Accuracy')
ax1.set_title('CNN - Training Accuracy', fontsize=12, fontweight='bold')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(alpha=0.3)

# Plot 2: Emotion Model Training Loss
ax2 = plt.subplot(3, 3, 2)
ax2.plot(history_cnn.history['loss'], label='Train Loss')
ax2.plot(history_cnn.history['val_loss'], label='Validation Loss')
ax2.set_title('CNN - Training Loss', fontsize=12, fontweight='bold')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(alpha=0.3)

# Plot 3: Confusion Matrix
ax3 = plt.subplot(3, 3, 3)
cm = confusion_matrix(y_fer_test, y_pred_fer)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax3, cbar=False,
            xticklabels=emotion_labels, yticklabels=emotion_labels)
ax3.set_title('Emotion Classification - Confusion Matrix', fontsize=12, fontweight='bold')
ax3.set_ylabel('True')
ax3.set_xlabel('Predicted')

# Plot 4: LSTM Trajectory Loss
ax4 = plt.subplot(3, 3, 4)
ax4.plot(lstm_history.history['trajectory_prediction_loss'], label='Trajectory Loss')
ax4.plot(lstm_history.history['val_trajectory_prediction_loss'], label='Val Trajectory Loss')
ax4.set_title('LSTM - Trajectory Prediction Loss', fontsize=12, fontweight='bold')
ax4.set_xlabel('Epoch')
ax4.set_ylabel('Loss (MSE)')
ax4.legend()
ax4.grid(alpha=0.3)

# Plot 5: LSTM Emotion Loss
ax5 = plt.subplot(3, 3, 5)
ax5.plot(lstm_history.history['emotion_prediction_loss'], label='Emotion Loss')
ax5.plot(lstm_history.history['val_emotion_prediction_loss'], label='Val Emotion Loss')
ax5.set_title('LSTM - Emotion Auxiliary Loss', fontsize=12, fontweight='bold')
ax5.set_xlabel('Epoch')
ax5.set_ylabel('Loss')
ax5.legend()
ax5.grid(alpha=0.3)

# Plot 6: Frame Processing Time
ax6 = plt.subplot(3, 3, 6)
processing_times_ms = [r['processing_time_ms'] for r in test_results]
ax6.plot(processing_times_ms, marker='o', linestyle='-', linewidth=2, markersize=6)
ax6.axhline(y=100, color='r', linestyle='--', label='Real-time Threshold (100ms)')
ax6.set_title('Frame Processing Time Over Time', fontsize=12, fontweight='bold')
ax6.set_xlabel('Frame Number')
ax6.set_ylabel('Processing Time (ms)')
ax6.legend()
ax6.grid(alpha=0.3)

# Plot 7: Emotion Distribution
ax7 = plt.subplot(3, 3, 7)
emotion_counts = pd.Series([r['emotion_label'] for r in test_results]).value_counts()
colors = plt.cm.Set3(np.linspace(0, 1, len(emotion_counts)))
ax7.barh(emotion_counts.index, emotion_counts.values, color=colors)
ax7.set_title('Detected Emotions Distribution', fontsize=12, fontweight='bold')
ax7.set_xlabel('Count')

# Plot 8: Confidence Distribution
ax8 = plt.subplot(3, 3, 8)
ax8.hist(confidences, bins=15, color='skyblue', edgecolor='black', alpha=0.7)
ax8.axvline(x=np.mean(confidences), color='r', linestyle='--', label=f'Mean: {np.mean(confidences):.3f}')
ax8.set_title('Emotion Confidence Distribution', fontsize=12, fontweight='bold')
ax8.set_xlabel('Confidence')
ax8.set_ylabel('Frequency')
ax8.legend()
ax8.grid(alpha=0.3, axis='y')

# Plot 9: Vehicle Actions Pie Chart
ax9 = plt.subplot(3, 3, 9)
action_counts = pd.Series(vehicle_actions).value_counts()
colors_actions = ['#ff6b6b', '#ffd93d', '#6bcf7f']
ax9.pie(action_counts.values, labels=action_counts.index, autopct='%1.1f%%',
        colors=colors_actions[:len(action_counts)], startangle=90)
ax9.set_title('Vehicle Action Distribution', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('emotion_trajectory_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Main visualization saved as 'emotion_trajectory_analysis.png'")

**Section 13: Video Frame Visualization with Overlays**

In [ ]:
print("\n" + "="*70)
print("VIDEO FRAME VISUALIZATION WITH OVERLAYS")
print("="*70)

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Real-time AV Perception: Emotion & Trajectory Visualization',
             fontsize=16, fontweight='bold', y=0.995)

sample_indices = [0, 3, 6, 9, 12, 14] if len(test_results) >= 15 else range(len(test_results))

for idx, sample_idx in enumerate(sample_indices):
    ax = axes[idx // 3, idx % 3]

    if sample_idx < len(test_results):
        result = test_results[sample_idx]

        # Create mock video frame
        frame = np.ones((480, 640, 3), dtype=np.uint8) * 200

        # Pedestrian bounding box
        ped_x, ped_y, ped_w, ped_h = 300, 150, 80, 200
        cv2.rectangle(frame, (ped_x - ped_w//2, ped_y - ped_h//2),
                      (ped_x + ped_w//2, ped_y + ped_h//2), (0, 255, 0), 3)

        # Emotion label with color
        emotion_color = {
            'Happy': (0, 255, 0),
            'Neutral': (255, 255, 0),
            'Sad': (0, 0, 255),
            'Angry': (0, 0, 128),
            'Disgust': (128, 0, 128),
            'Fear': (255, 0, 255),
            'Surprise': (0, 255, 255)
        }

        color = emotion_color.get(result['emotion_label'], (255, 255, 255))
        cv2.putText(frame, f"Emotion: {result['emotion_label']}", (20, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 2)
        cv2.putText(frame, f"Conf: {result['confidence']:.3f}", (20, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

        # Vehicle action
        action_color = {
            'BRAKE': (0, 0, 255),
            'CAUTION': (0, 165, 255),
            'PROCEED': (0, 255, 0)
        }
        action_col = action_color.get(result['vehicle_action'], (255, 255, 255))
        cv2.putText(frame, f"Action: {result['vehicle_action']}", (20, 400),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, action_col, 2)
        cv2.putText(frame, f"Speed: {result['speed_adjustment']:.1f} km/h", (20, 450),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

        # Predicted trajectory arrow
        cv2.arrowedLine(frame, (ped_x, ped_y), (ped_x + 80, ped_y + 50),
                        (255, 0, 0), 3, tipLength=0.3)
        cv2.putText(frame, "Predicted Path", (ped_x + 20, ped_y - 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        cv2.putText(frame, f"Frame {result['frame_number']}", (20, 150),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

        ax.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    ax.set_title(f"Frame {sample_idx + 1}", fontsize=11, fontweight='bold')
    ax.axis('off')

plt.tight_layout()
plt.savefig('av_perception_frames.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Frame visualization saved as 'av_perception_frames.png'")


**Section 14: Results Summary**

In [ ]:
print("\n" + "="*70)
print("FINAL RESULTS SUMMARY")
print("="*70)

print(f"""
{'='*70}
QUANTUM-ENHANCED EMOTION-AWARE AUTONOMOUS VEHICLE SYSTEM
COMPLETE PERFORMANCE REPORT
{'='*70}

📊 EMOTION RECOGNITION MODEL (CNN)
{'─'*70}
  Overall Accuracy:  {emotion_accuracy:.4f}
  Weighted Precision: {emotion_precision:.4f}
  Weighted Recall:    {emotion_recall:.4f}
  Weighted F1-Score:  {emotion_f1:.4f}

📊 LSTM TRAJECTORY PREDICTION MODEL
{'─'*70}
  Input Sequence Length: 30 frames
  Prediction Horizon: 5 frames
  Features per Frame: 4 (x, y, w, h)

⏱️ REAL-TIME PROCESSING STATISTICS
{'─'*70}
  Total Frames Processed: {stats['total_frames_processed']}
  Average Processing Time: {stats['avg_processing_time_ms']:.2f} ms
  Max Processing Time: {stats['max_processing_time_ms']:.2f} ms
  Min Processing Time: {stats['min_processing_time_ms']:.2f} ms
  Real-time Capability: {'✅ PASS' if stats['real_time_capability'] else '❌ FAIL'} (Threshold: <100ms)
  Frames Per Second: {1000 / stats['avg_processing_time_ms']:.1f} FPS

🧠 QUANTUM-INSPIRED DECISION ENGINE
{'─'*70}
  Decision Logic: Probabilistic superposition with amplitude-based measurement
  Ambiguous Cases Handled: {sum(1 for r in test_results if r['confidence'] < 0.7)} frames
  Average Confidence: {np.mean(confidences):.4f}
  High Confidence (>0.7): {sum(1 for c in confidences if c > 0.7)} frames
  Medium Confidence (0.5-0.7): {sum(1 for c in confidences if 0.5 <= c <= 0.7)} frames
  Low Confidence (<0.5): {sum(1 for c in confidences if c < 0.5)} frames

🚗 VEHICLE ACTION RESPONSE
{'─'*70}
""")

for action, count in action_counts.items():
    print(f"  {action:8} | {count:2d} frames ({100*count/len(vehicle_actions):5.1f}%)")

print(f"""
📈 EMOTION-ACTION MAPPING
{'─'*70}
  Angry      → BRAKE        (Stop immediately - high risk)
  Disgust    → BRAKE        (Stop immediately - high risk)
  Fear       → BRAKE        (Stop immediately - high risk)
  Happy      → PROCEED      (Safe pedestrian - continue)
  Neutral    → PROCEED      (Safe pedestrian - continue)
  Sad        → CAUTION      (Reduced speed - 60% speed)
  Surprise   → CAUTION      (Reduced speed - 60% speed)

✅ DATASETS USED
{'─'*70}
  FER-2013:
    - Samples used: {len(fer2013_df)}
    - Train/Test: {len(X_fer_train)} / {len(X_fer_test)}
    - Image size: 48×48 grayscale
    - Emotions: 7 classes

  PIE (Pedestrian Trajectory):
    - Sequences: {len(pie_df)}
    - Frames per sequence: 30
    - Features: x, y, width, height
    - Emotions: 7 classes

🔬 QUANTUM THEORY PRINCIPLES APPLIED
{'─'*70}
  ✓ Superposition: Emotional states as probabilistic amplitudes
  ✓ Measurement: |amplitude|² determines final action probability
  ✓ Uncertainty: Low confidence → increased superposition (softer decisions)
  ✓ Real-world Validation: Handles ambiguous pedestrian behaviors

📝 CONCLUSION
{'─'*70}
The implemented system successfully demonstrates:
  ✓ Real-time facial emotion recognition (CNN accuracy: {emotion_accuracy:.1%})
  ✓ LSTM-based spatio-temporal trajectory prediction
  ✓ Quantum-inspired probabilistic decision logic
  ✓ Adaptive vehicle response (speed & trajectory adjustment)
  ✓ Real-time processing capability ({stats['avg_processing_time_ms']:.1f} ms per frame)

This system can improve pedestrian safety in autonomous vehicles by:
  • Detecting pedestrian emotional states from facial expressions
  • Predicting pedestrian trajectory patterns
  • Making robust decisions even with ambiguous signals
  • Adapting vehicle behavior for pedestrian safety

{'='*70}
""")

print("✅ ANALYSIS COMPLETE!")
print(f"Generated visualizations:")
print(f"  - emotion_trajectory_analysis.png")
print(f"  - av_perception_frames.png")